# Содержание:

### [ABC анализ](#abcanalysis) 
### [XYZ анализ](#xyzanalysis)
### [Динамика продаж](#динамикапродаж)

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', 40)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
data = pd.read_csv('data1.csv', encoding='1251')

In [4]:
# Переименуем столбцы для улучшения читаемости кода
data = data.rename(columns={'DR_Kol':'Quantity', 'DR_NDrugs':'Name', 'DR_Dat':'Date', 
                            'DR_CRoz':'Retail_Price', 'DR_CZak': 'Purchase_Price', 'DR_SDisc':'Discount'})

In [1]:
# <a id="abcanalysis"></a>

## abcanalysis

# ABC analysis

In [5]:
data['Revenue'] = data['Retail_Price'] * data['Quantity']
data['Margin'] = data['Retail_Price'] - data['Purchase_Price'] / data['Purchase_Price']

In [6]:
df = data[['Name', 'Quantity', 'Revenue', 'Margin']]
df.head(3)

,Name,Quantity,Revenue,Margin
0,ГАСТАЛ №12 ТАБ. Д/РАСС.,1.00,270.00,269.00
1,"ТОБРОПТ 0,3% 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП. /РОМФАРМ/",1.00,127.00,126.00
2,ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-М...,1.00,2563.00,2562.00


In [7]:
def perform_abc(df, index):
    cols = list(df.columns)
    cols.remove(index)
    groupped_df = df.groupby(index).agg({'Quantity': 'sum', 'Revenue':'max', 'Margin':'max'})
    for col in cols:
        groupped_df[f'rel_{col}'] = groupped_df[col] / sum(groupped_df[col])
        groupped_df = groupped_df.sort_values(f'rel_{col}', ascending=False)
        groupped_df[f'cumsum_{col}'] = groupped_df[f'rel_{col}'].cumsum()
        groupped_df[f'ABC_{col}'] = np.where(groupped_df[f'cumsum_{col}'] < 0.8, 'A', np.where(groupped_df[f'cumsum_{col}'] < 0.95, 'B', 'C'))
    return groupped_df[cols + [f'ABC_{col}' for col in cols]]

In [8]:
abc_df = perform_abc(df, index='Name')
abc_df

,Quantity,Revenue,Margin,ABC_Quantity,ABC_Revenue,ABC_Margin
Name,,,,,,
КСАРЕЛТО 10МГ. №98 ТАБ. П/П/О /БАЙЕР/,0.14,1539.39,10774.70,C,A,A
ПРАДАКСА 110МГ. №180 КАПС. /БЕРИНГЕР/,0.33,3458.33,10374.00,C,A,A
ГЕЛАДРИНК ФОРТЕ ПОР. Д/ПРИЕМА ВНУТРЬ АПЕЛЬСИН 420Г. БАНКА,1.00,4985.00,4984.00,B,A,A
ХАРТМАНН БРАНОЛИНД H ПОВЯЗКА СТЕР. 10Х20СМ. №30 ПЕРУАН.БАЛЬЗАМ /АРТ.4923462/ [BRANOLIND],0.07,273.00,4094.00,C,A,A
ОМРОН ТОНОМЕТР M2 БАЗИК АВТОМАТ АДАПТЕР /АРТ.HEM-7121-ARU/ [OMRON],1.00,3832.00,3831.00,A,A,A
...,...,...,...,...,...,...
ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 6X10 №1 /ВЕРОФАРМ/,13.00,80.00,7.00,A,C,C
"НАФТИЗИН 0,1% 15МЛ. НАЗАЛ.КАПЛИ ФЛ./КАП. /СЛАВЯНСКАЯ АПТЕКА/",35.00,96.00,7.00,A,C,C
"ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 2,5Х7,2 №1 /ВЕРОФАРМ/",68.00,96.00,3.00,A,C,C


In [9]:
abc_df['ABC'] = abc_df[['ABC_Quantity', 'ABC_Revenue', 'ABC_Margin']].agg(''.join, axis=1)
abc_df

,Quantity,Revenue,Margin,ABC_Quantity,ABC_Revenue,ABC_Margin,ABC
Name,,,,,,,
КСАРЕЛТО 10МГ. №98 ТАБ. П/П/О /БАЙЕР/,0.14,1539.39,10774.70,C,A,A,CAA
ПРАДАКСА 110МГ. №180 КАПС. /БЕРИНГЕР/,0.33,3458.33,10374.00,C,A,A,CAA
ГЕЛАДРИНК ФОРТЕ ПОР. Д/ПРИЕМА ВНУТРЬ АПЕЛЬСИН 420Г. БАНКА,1.00,4985.00,4984.00,B,A,A,BAA
ХАРТМАНН БРАНОЛИНД H ПОВЯЗКА СТЕР. 10Х20СМ. №30 ПЕРУАН.БАЛЬЗАМ /АРТ.4923462/ [BRANOLIND],0.07,273.00,4094.00,C,A,A,CAA
ОМРОН ТОНОМЕТР M2 БАЗИК АВТОМАТ АДАПТЕР /АРТ.HEM-7121-ARU/ [OMRON],1.00,3832.00,3831.00,A,A,A,AAA
...,...,...,...,...,...,...,...
ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 6X10 №1 /ВЕРОФАРМ/,13.00,80.00,7.00,A,C,C,ACC
"НАФТИЗИН 0,1% 15МЛ. НАЗАЛ.КАПЛИ ФЛ./КАП. /СЛАВЯНСКАЯ АПТЕКА/",35.00,96.00,7.00,A,C,C,ACC
"ЛЕЙКОПЛАСТЫРЬ БАКТЕР. 2,5Х7,2 №1 /ВЕРОФАРМ/",68.00,96.00,3.00,A,C,C,ACC


<a id='xyzanalysis'></a>

# XYZ analysis

In [10]:
df

,Date,Name,Quantity
0,2022-08-01,"911-ВЕНОЛГОН ГЕЛЬ Д/НОГ ПРИ ТЯЖЕСТИ,БОЛИ,ОТЕКА...",1.00
1,2022-08-01,L-ТИРОКСИН 100МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,1.00
2,2022-08-01,L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,1.00
3,2022-08-01,L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,2.00
4,2022-08-01,NF ВАТА ХИРУРГ. СТЕР. 100Г. /НЬЮФАРМ/,1.00
...,...,...,...
1084,2022-08-05,"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",1.00
1085,2022-08-05,ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,2.00
1086,2022-08-05,ЭСВИЦИН СР-ВО П/ОБЛЫСЕНИЯ ЛОСЬОН-ТОНИК 250МЛ. ...,1.00
1087,2022-08-05,ЭТОРИАКС 90МГ. №7 ТАБ. П/П/О,1.00


In [11]:
# Отфильтруем товары, которые продавались более одного раза (продавались несколько дней)
df1 = df.groupby(['Name']).agg({'Date':'count'}).reset_index()
xyz_names = list(df1[df1['Date'] > 1]['Name'])
df = df[df['Name'].isin(xyz_names)]

In [12]:
# Рассчитаем коэффициент вариации
df = df.groupby(['Name']).apply(lambda x: x.select_dtypes(include='number').std() / x.select_dtypes(include='number').mean())
df

,Quantity
Name,
L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,0.00
L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,0.47
АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,0.00
АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,0.47
АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/ФАРМСТАНДАРТ/,0.43
...,...
"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",0.47
ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-МАЙЕРС/,0.00
ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,0.47


In [13]:
# Переименуем столбец Coefficient of variation
df = df.rename(columns={'Quantity': 'CV'})
df

,CV
Name,
L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,0.00
L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,0.47
АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,0.00
АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,0.47
АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/ФАРМСТАНДАРТ/,0.43
...,...
"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",0.47
ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-МАЙЕРС/,0.00
ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,0.47


In [14]:
# В зависимости от коэффициента вариации определим группу товара
df['XYZ'] = np.where(df['CV'] < 0.1, 'X', np.where(df['CV'] < 0.25, 'Y', 'Z'))

In [15]:
df

,CV,XYZ
Name,,
L-ТИРОКСИН 50МКГ. №50 ТАБ. /БЕРЛИН ХЕМИ/,0.00,X
L-ТИРОКСИН 75МКГ. №100 ТАБ. /БЕРЛИН ХЕМИ/,0.47,Z
АЗАРГА 10МГ/МЛ.+5МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ./КАП.,0.00,X
АМОКСИЦИЛЛИН 500МГ. №16 КАПС. /ХЕМОФАРМ/,0.47,Z
АРБИДОЛ МАКСИМУМ 200МГ. №10 КАПС. /ОТИСИФАРМ/ФАРМСТАНДАРТ/,0.43,Z
...,...,...
"ЭВО ПАНТЕНОЛ ПОМАДА ГИГИЕН. 2,8Г. [EVO] /АВАНТА/",0.47,Z
ЭЛИКВИС 5МГ. №60 ТАБ. П/П/О /ПФАЙЗЕР/БРИСТОЛ-МАЙЕРС/,0.00,X
ЭМОКСИПИН 10МГ/МЛ. 5МЛ. №1 ГЛ.КАПЛИ ФЛ. КРЫШ/КАП.,0.47,Z


<a id='динамикапродаж'></a>

## динамикапродаж

In [16]:
df = data.groupby('Date')[['Quantity', 'Retail_Price', 'Discount']
                         ].apply(lambda x: pd.Series({'Revenue': sum(x['Quantity']*x['Retail_Price'] - x['Discount'])})).reset_index()
df

,Date,Revenue
0,2022-08-01,84681.52
1,2022-08-02,71389.37
2,2022-08-03,78050.82
3,2022-08-04,59187.36
4,2022-08-05,56458.81


In [17]:
# Вариант 1
df['Revenue_d'] = df['Revenue'].rolling(2).apply(lambda x: (x.iloc[1] - x.iloc[0])/x.iloc[0])
df

,Date,Revenue,Revenue_d
0,2022-08-01,84681.52,NaN
1,2022-08-02,71389.37,-0.16
2,2022-08-03,78050.82,0.09
3,2022-08-04,59187.36,-0.24
4,2022-08-05,56458.81,-0.05


In [18]:
# Вариант 2
df['Revenue_Shifted'] = df['Revenue'].shift(1)
df['Revenue_d'] = (df['Revenue'] - df['Revenue_Shifted']) / df['Revenue_Shifted']
df

,Date,Revenue,Revenue_d,Revenue_Shifted
0,2022-08-01,84681.52,NaN,NaN
1,2022-08-02,71389.37,-0.16,84681.52
2,2022-08-03,78050.82,0.09,71389.37
3,2022-08-04,59187.36,-0.24,78050.82
4,2022-08-05,56458.81,-0.05,59187.36


In [19]:
# Вариант 3
df['Revenue_d'] = df['Revenue'].pct_change(1)
df

,Date,Revenue,Revenue_d,Revenue_Shifted
0,2022-08-01,84681.52,NaN,NaN
1,2022-08-02,71389.37,-0.16,84681.52
2,2022-08-03,78050.82,0.09,71389.37
3,2022-08-04,59187.36,-0.24,78050.82
4,2022-08-05,56458.81,-0.05,59187.36
